In [0]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
root_dir = "/content/gdrive/My Drive/Deep Learning project/"
image_dir = root_dir + 'Images/'

Mounted at /content/gdrive/


In [0]:
# To determine which version you're using:
!pip show tensorflow

Name: tensorflow
Version: 2.2.0rc4
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: gast, numpy, protobuf, absl-py, h5py, tensorboard, tensorflow-estimator, wrapt, six, scipy, keras-preprocessing, termcolor, grpcio, astunparse, google-pasta, opt-einsum, wheel
Required-by: fancyimpute


In [0]:
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img, save_img

import imageio
import imgaug as ia

import os
import numpy as np
from numpy import expand_dims

import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display 
from PIL import Image
import h5py
%matplotlib inline

In [0]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.50.8.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.50.8.138:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:
train_source = image_dir + 'Split/Train/'
test_source = image_dir + 'Split/Test/'

In [0]:
# Data Augmentation
train_dataGenerator = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   rotation_range=20, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest', 
                                   validation_split=0.2) # set validation split

In [0]:
test_dataGenerator = ImageDataGenerator(rescale = 1./255)

In [0]:
# Default Values

#img_height = 150
#img_width = 150
batch_size = 32

In [0]:
train_set = train_dataGenerator.flow_from_directory(train_source,
                                                 target_size = (64, 64),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

test_set = test_dataGenerator.flow_from_directory(test_source,
                                            target_size = (64, 64),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

Found 3037 images belonging to 9 classes.
Found 949 images belonging to 9 classes.


In [0]:
# Writing a CNN Model

image_classifier = Sequential()
image_classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
image_classifier.add(MaxPooling2D(pool_size = (2, 2)))
image_classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
image_classifier.add(MaxPooling2D(pool_size = (2, 2)))
image_classifier.add(Flatten())
image_classifier.add(Dense(units = 128, activation = 'relu'))
image_classifier.add(Dense(units = 9, activation = 'softmax')) # number of classes
image_classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
image_classifier.fit_generator(train_set,
                               steps_per_epoch = 3037,
                               epochs = 25,
                               validation_data = test_set,
                               validation_steps = 949) 


Epoch 1/25
  47/3037 [..............................] - ETA: 6:29:25 - loss: 2.1600 - accuracy: 0.1672

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 55 could not be retrieved. It could be because a worker has died.
  UserWarning)


3037/3037 [==============================] - 1517s 500ms/step - loss: 1.4715 - accuracy: 0.4960 - val_loss: 0.7009 - val_accuracy: 0.6585
Epoch 2/25
3037/3037 [==============================] - 591s 195ms/step - loss: 1.0544 - accuracy: 0.6472 - val_loss: 0.8383 - val_accuracy: 0.6736
Epoch 3/25
3037/3037 [==============================] - 610s 201ms/step - loss: 0.8869 - accuracy: 0.7031 - val_loss: 1.2415 - val_accuracy: 0.7131
Epoch 4/25
3037/3037 [==============================] - 633s 209ms/step - loss: 0.7863 - accuracy: 0.7345 - val_loss: 1.4412 - val_accuracy: 0.7084
Epoch 5/25
3037/3037 [==============================] - 656s 216ms/step - loss: 0.7150 - accuracy: 0.7599 - val_loss: 1.4971 - val_accuracy: 0.7090
Epoch 6/25
3037/3037 [==============================] - 634s 209ms/step - loss: 0.6607 - accuracy: 0.7783 - val_loss: 0.9026 - val_accuracy: 0.6975
Epoch 7/25
3037/3037 [==============================] - 622s 205ms/step - loss: 0.6139 - accuracy: 0.7928 - val_loss: 1.91

In [0]:
# Part 3 - Making new predictions
import numpy as np
from keras.preprocessing import image

sampleImage = test_source+'Planks/Planks55-0.jpg'
# load the image
img = load_img(sampleImage, target_size = (64,64))
data = img_to_array(img)
test_image = np.expand_dims(data, axis = 0)
result = image_classifier.predict(test_image)
print(train_set.class_indices)


{'DownwardFacingDog': 0, 'LowLunge': 1, 'Planks': 2, 'ReversePlanks': 3, 'SeatedForwardBend': 4, 'SidePlanks': 5, 'TreePose': 6, 'TrianglePose': 7, 'WarriorPose': 8}


In [0]:
if result[0][0] == 1:
	prediction = 'downwardFacingDog' #predictions in array are in alphabetical order
elif result[0][1] == 1:
	prediction = 'LowLung'
elif result[0][2] == 1:
	prediction = 'planks'
elif result[0][3] == 1:
	prediction = 'reversePlank'
elif result[0][4] == 1:
	prediction = 'seatdForwardBend'
elif result[0][5] == 1:
	prediction = 'sidePlanks'
elif result[0][6] == 1:
	prediction = 'treePose'
elif result[0][7] == 1:
	prediction = 'trianglepose'
elif result[0][8] == 1:
	prediction = 'warrior'


print(result)
print(prediction)

[[0. 0. 0. 0. 0. 1. 0. 0. 0.]]
sidePlanks
